<a href="https://colab.research.google.com/github/Ochodus/DBLab_onehot/blob/main/One_Hot_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
import pandas as pd
import numpy as np
import random
from google.colab.data_table import DataTable

In [25]:
df = pd.read_excel('drive/MyDrive/DBLab_Onehot/mental_health_medical_data.xlsx')
DataTable.max_columns = 200

In [108]:
# 질병 목록 추출
def getDiseases(df):
  return df['disease']

#증상 목록 추출
def getSymptoms(df):
  symptoms = []
  del_token = "[']"
  x = ""
                     
  for s in df['symptoms']:
    for t in del_token:
      s = s.replace(t, "")
    s = s.split(',')
    for i in s:
      if len(i) > 0 and i[0] == " ":
        i = i[1:]
      if i in symptoms or i == "":
        continue
      else:
        symptoms.append(i)

  return symptoms

#One-hot 인코딩
def createOneHot(df, diseases, symptoms):
  oneHot = pd.DataFrame()
  for symp in symptoms:
    oneHot.loc[:,symp] = [0] * len(diseases)
  oneHot.set_index(keys=diseases, inplace=True)

  for disease, symps in zip(df['disease'], df['symptoms']):
    for symptom in symptoms:
      if "'"+symptom+"'" in symps:
        oneHot.loc[disease, symptom] = 1

  return oneHot

#하나의 질병에서만 나타나는 증상 목록 구하기
def getUnqSym(oneHot, diseases, symptoms):
  unqSym = []
  for symptom in symptoms:
    count = 0
    first_index = -1
    for i, j in enumerate(oneHot[symptom]):
      if j == 1:
        count += 1
        if count == 1:
          first_index = i
    if count == 1:
      unqSym.append([symptom, diseases[first_index]])

  return unqSym

#각 질병 별 증상 갯수 구하기
def getSymNum(oneHot, diseases):
  symNum = []
  for disease in diseases:
    sum = 0
    for i in oneHot.loc[disease]:
      sum += i
    symNum.append([disease, sum])
    
  return symNum

#질병의 평균 증상 갯수 구하기
def getAvgSymNum(symNum, diseases):
  total = 0
  for num in symNum:
    total += num[1]
  avgSymNum = total/len(diseases)
  return avgSymNum

In [115]:
#테스트 벡터 생성(현재 임시로 random 함수를 이용해 증상 생성)
def createUsrVec(symptoms):
  usrVec = pd.DataFrame(columns=symptoms)
  usrVec.loc[0] = [0] * len(symptoms)
  rndSym = random.randrange(1, len(symptoms) + 1)
  usrVec.iloc[0, rndSym] = 1
  return usrVec

#테스트 벡터와 oneHot Matrix의 곱 구하기
def multUsrVec(oneHot, usrVec):
  mult = usrVec.to_numpy() * oneHot.to_numpy()
  mult = pd.DataFrame(mult, columns=symptoms)
  mult.set_index(keys=diseases, inplace=True)
  return mult

In [124]:
diseases = getDiseases(df)
symptoms = getSymptoms(df)

oneHot = createOneHot(df, diseases, symptoms)

unqSym = getUnqSym(oneHot, diseases, symptoms)
symNum = getSymNum(oneHot, diseases)
avgSymNum = getAvgSymNum(symNum, diseases)

unqSym = pd.DataFrame(unqSym, columns = ['Symptom', 'Disease'])
symNum = pd.DataFrame(symNum, columns = ['Disease','Symptom Amount'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = value


In [125]:
unqSym

,Symptom,Disease
0,반복적인 동작,강박 장애
1,숨막히는 느낌,공황 장애
2,가슴 통증,공황 장애
3,죽음에 대한 공포,공황 장애
4,오한,공황 장애
...,...,...
106,운동장애,레트 증후군
107,좁은 공간에서의 불안감,폐소공포증
108,흉부압박감,과호흡 증후군
109,부정맥,과호흡 증후군


In [126]:
symNum

,Disease,Symptom Amount
0,강박 장애,3
1,경계성 인격장애,7
2,공황 장애,13
3,금단 현상,13
4,분열정동 장애,8
...,...,...
59,혈관성 치매,4
60,히스테리성 인격장애,0
61,레트 증후군,4
62,폐소공포증,2


In [117]:
oneHot

,불안,반복적인 동작,의심,약물남용,대인관계 어려움,조절할 수 없는 충동감,자살의 위험,우울,망상,숨막히는 느낌,...,이인증,정체감 혼란 및 변형,위험한 행동 반복,정서 및 행동의 일탈,마비,운동장애,좁은 공간에서의 불안감,흉부압박감,부정맥,호흡곤란
disease,,,,,,,,,,,,,,,,,,,,,
강박 장애,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
경계성 인격장애,0,0,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
공황 장애,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
금단 현상,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
분열정동 장애,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
혈관성 치매,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
히스테리성 인격장애,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
레트 증후군,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [118]:
usrVec = createUsrVec(symptoms)
usrVec

,불안,반복적인 동작,의심,약물남용,대인관계 어려움,조절할 수 없는 충동감,자살의 위험,우울,망상,숨막히는 느낌,...,이인증,정체감 혼란 및 변형,위험한 행동 반복,정서 및 행동의 일탈,마비,운동장애,좁은 공간에서의 불안감,흉부압박감,부정맥,호흡곤란
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
mult = multUsrVec(oneHot, usrVec)
mult

,불안,반복적인 동작,의심,약물남용,대인관계 어려움,조절할 수 없는 충동감,자살의 위험,우울,망상,숨막히는 느낌,...,이인증,정체감 혼란 및 변형,위험한 행동 반복,정서 및 행동의 일탈,마비,운동장애,좁은 공간에서의 불안감,흉부압박감,부정맥,호흡곤란
disease,,,,,,,,,,,,,,,,,,,,,
강박 장애,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
경계성 인격장애,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
공황 장애,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
금단 현상,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
분열정동 장애,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
혈관성 치매,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
히스테리성 인격장애,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
레트 증후군,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
